__Kaggle competition - house prices__

1. [Import](#Import)
    1. [Tools](#Tools)
    1. [Data](#Data)    
1. [EDA](#EDA)
    1. [Category feature EDA](#Category-feature-EDA)
    1. [Count feature EDA](#Count-feature-EDA)
    1. [Continuous feature EDA](#Continuous-feature-EDA)
    1. [Faceting](#Faceting)
    1. [Target variable evaluation](#Target-variable-evaluation)    
1. [Data preparation](#Data-preparation)
    1. [Missing data](#Missing-data)
    1. [Feature engineering](#Feature-engineering)
        1. [Handcrafted](#Handcrafted)
        1. [Polynomial features](#Polynomial-features)
        1. [Encoding](#Encoding)
    1. [Feature transformation](#Feature-transformation)
        1. [Skew correction](#Skew-correction)
        1. [Scaling](#Scaling)     
    1. [Outliers](#Outliers)
    1. [Additional exploratory data analysis](#Additional-exploratory-data-analysis)
1. [Feature importance](#Feature-importance)    
1. [Modeling](#Modeling)
    1. [Data preparation](#Data-preparation-1)
    1. [Bayesian hyper-parameter optimization](#Bayesian-hyper-parameter-optimization)
    1. [Model performance evaluation - standard models](#Model-performance-evaluation-standard-models)
    1. [Model explanability](#Model-explanability)
    1. [Submission - standard models](#Submission-standard-models)
1. [Stacking](#Stacking)
    1. [Primary models](#Primary-models)
    1. [Meta model](#Meta-model)                
    1. [Model performance evaluation - stacked models](#Model-performance-evaluation-stacked-models)
    1. [Submission - stacked models](#Submission-stacked-models)    

# Import

<a id = 'Import'></a>

## Tools

<a id = 'Tools'></a>

In [ ]:
# standard libary and settings
import copy
import os
import pickle
import sys
import importlib
import itertools
from functools import reduce
import time

rundate = time.strftime("%Y%m%d")

import warnings

warnings.simplefilter("ignore")

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

# data extensions and settings
import numpy as np

np.set_printoptions(threshold=np.inf, suppress=True)

import pandas as pd

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.options.display.float_format = "{:,.6f}".format

# modeling extensions
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    ExtraTreesRegressor,
    IsolationForest,
)
from sklearn.kernel_ridge import KernelRidge
from sklearn.impute import SimpleImputer
from sklearn.linear_model import (
    Lasso,
    Ridge,
    ElasticNet,
    LinearRegression,
    SGDRegressor,
)
from sklearn.model_selection import (
    KFold,
    train_test_split,
    GridSearchCV,
    StratifiedKFold,
    cross_val_score,
    RandomizedSearchCV,
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    PolynomialFeatures,
    OrdinalEncoder,
    LabelEncoder,
    OneHotEncoder,
    KBinsDiscretizer,
    QuantileTransformer,
    PowerTransformer,
    MinMaxScaler,
)
from sklearn.svm import SVR
from category_encoders import (
    WOEEncoder,
    TargetEncoder,
    CatBoostEncoder,
    BinaryEncoder,
    CountEncoder,
)

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from hyperopt import hp

import eif
import shap

shap.initjs()

# visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import missingno as msno
import squarify

sys.path.append(f"{os.environ['REPOS']}/mlmachine")
sys.path.append(f"{os.environ['REPOS']}/prettierplot")

import mlmachine as mlm
import mlmachine.data as data
from mlmachine.features.preprocessing import (
    DataFrameSelector,
    PandasTransformer,
    KFoldEncoder,
    GroupbyImputer,
    PandasFeatureUnion,
    DualTransformer,
)
from prettierplot.plotter import PrettierPlot
import prettierplot.style as style

%load_ext autoreload
%autoreload 2

## Reload objects

In [ ]:
# #

# experiment_path_root = "/data/t1-tpeterso/repos/kaggle-housing-prices/experiments/housing_prices"
# experiment = ""

# # reload objects
# machine = pickle.load(open(os.path.join(experiment_path_root, experiment, "machine", "machine.pkl"), 'rb'))
# impute_pipe = pickle.load(open(os.path.join(experiment_path_root, experiment, "transformers", "impute_pipe.pkl"), 'rb'))
# polynomial_pipe = pickle.load(open(os.path.join(experiment_path_root, experiment, "transformers", "polynomial_pipe.pkl"), 'rb'))
# encode_pipe = pickle.load(open(os.path.join(experiment_path_root, experiment, "transformers", "encode_pipe.pkl"), 'rb'))
# target_encode_pipe = pickle.load(open(os.path.join(experiment_path_root, experiment, "transformers", "target_encode_pipe.pkl"), 'rb'))
# skew_pipe = pickle.load(open(os.path.join(experiment_path_root, experiment, "transformers", "skew_pipe.pkl"), 'rb'))
# scale_pipe = pickle.load(open(os.path.join(experiment_path_root, experiment, "transformers", "scale_pipe.pkl"), 'rb'))
# fs = pickle.load(open(os.path.join(experiment_path_root, experiment, "feature_selection", "FeatureSelector.pkl"), 'rb'))


## Data

### Load & review dataset

In [ ]:
# load data and print dimensions
df_train, df_valid = data.housing()

print("Training data dimensions: {}".format(df_train.shape))
print("Validation data dimensions: {}".format(df_valid.shape))

In [ ]:
# display info and first 5 rows
df_train.info()
display(df_train[:5])

In [ ]:
# review counts of different column types
df_train.dtypes.value_counts()


### Create machine object

In [ ]:
continuous = [
    "LotFrontage",
    "LotArea",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "LowQualFinSF",
    "GrLivArea",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch",
    "PoolArea",
]

remove_features = [
    "Id",
    "MiscVal",
]

count = [
    "BsmtFullBath",
    "BsmtHalfBath",
    "FullBath",
    "HalfBath",
    "BedroomAbvGr",
    "KitchenAbvGr",
    "TotRmsAbvGrd",
    "Fireplaces",
    "GarageCars",
]

nominal = [
    "MSSubClass",
    "MSZoning",    
    "LandContour",
    "Neighborhood",
    "Condition1",
    "Condition2",
    "BldgType",
    "HouseStyle",
#     "YearBuilt",
#     "YearRemodAdd",
    "RoofStyle",
    "RoofMatl",
    "Exterior1st",
    "Exterior2nd",
    "MasVnrType",
    "Foundation",
    "Heating",
    "GarageType",
#     "GarageYrBlt", 
    "Fence",
    "SaleType",
    "SaleCondition",
#     "MiscFeature",    
#     "MoSold",
#     "YrSold",
    
]

ordinal = [
    "Street",  
    "Alley" ,
    "LotShape", 
    "Utilities", 
    "LotConfig",
    "LandSlope",
    "OverallQual",
    "OverallCond",    
    "ExterQual", 
    "ExterCond", 
    "BsmtQual", 
    "BsmtCond", 
    "BsmtExposure", 
    "BsmtFinType1", 
    "BsmtFinType2", 
    "HeatingQC", 
    "CentralAir", 
    "Electrical", 
    "KitchenQual", 
    "Functional", 
    "FireplaceQu", 
    "GarageFinish", 
    "GarageQual", 
    "GarageCond", 
    "PavedDrive", 
    "PoolQC",    
]

ordinal_encodings = {
    "Street": ["Grvl", "Pave"],
    "Alley": ["Nonexistent", "Grvl", "Pave"],
    "LotShape": ["IR3", "IR2", "IR1", "Reg"],
    "Utilities": ["ELO", "NoSeWa", "NoSewr", "AllPub"],
    "LotConfig": ["FR3", "FR2", "Corner", "Inside", "CulDSac"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "OverallQual": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "OverallCond": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "ExterQual": ["Po", "Fa", "TA", "Gd", "Ex"], 
    "ExterCond": ["Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtQual": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtCond": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtExposure": ["Nonexistent", "No", "Mn", "Av", "Gd"],
    "BsmtFinType1": ["Nonexistent", "Unf", "LwQ", "BLQ", "Rec", "ALQ", "GLQ"],
    "BsmtFinType2": ["Nonexistent", "Unf", "LwQ", "BLQ", "Rec", "ALQ", "GLQ"],
    "HeatingQC": ["Po", "Fa", "TA", "Gd", "Ex"],
    "CentralAir": ["N", "Y"],
    "Electrical": ["FuseP", "FuseF", "FuseA", "Mix", "SBrkr"],
    "KitchenQual": ["Po", "Fa", "TA", "Gd", "Ex"],
    "Functional": ["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"],
    "FireplaceQu": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "GarageFinish": ["Nonexistent", "Unf", "RFn", "Fin"],
    "GarageQual": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "GarageCond": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "PavedDrive": ["N", "P", "Y"],
    "PoolQC": ["Nonexistent", "Fa", "TA", "Gd", "Ex"],
}

In [ ]:
#
df_train, df_valid = mlm.train_test_df_compile(data=df_train, target_col="SalePrice")

machine = mlm.Machine(
    experiment_name="housing_price_regression",
    training_dataset=df_train,
    validation_dataset=df_valid,    
    target="SalePrice",
    remove_features=remove_features,
    identify_as_continuous=continuous,
    identify_as_count=count,    
    identify_as_nominal=nominal,
    identify_as_ordinal=ordinal,
    ordinal_encodings=ordinal_encodings,
    is_classification=False,
)


In [ ]:
# review mlm dtypes
machine.training_features.mlm_dtypes


# EDA

## Category feature EDA

In [ ]:
# number features
for feature in machine.training_features.mlm_dtypes["category"]:
    machine.eda_num_target_cat_feat(
        feature=feature,
        level_count_cap=10,
        chart_scale=20,
        training_data=True
    )


## Count feature EDA

In [ ]:
# number features
for feature in machine.training_features.mlm_dtypes["count"]:
    machine.eda_num_target_cat_feat(
        feature=feature,
#         level_count_cap=20,
        chart_scale=20,
        training_data=True,
    )


## Continuous feature EDA

In [ ]:
# continuous features
for feature in machine.training_features.mlm_dtypes["continuous"]:
    machine.eda_num_target_num_feat(
        feature=feature,
#         outliers_out_of_scope=5,
        chart_scale=20,
        training_data=True
    )


### Correlation

In [ ]:
# correlation heat map
p = PrettierPlot(chart_scale=25)
ax = p.make_canvas()
p.corr_heatmap(df=machine.training_features, ax=ax)

In [ ]:
# correlation heat map with most highly correlated features relative to the target
p = PrettierPlot(plot_orientation='tall',chart_scale=15)
ax = p.make_canvas()
p.corr_heatmap_target(df=machine.training_features, target=machine.training_target, thresh=0.6, annot = True, ax=ax)

### Pair plot

In [ ]:
# pair plot
p = PrettierPlot(chart_scale=10)
p.pair_plot(
    df=machine.training_features,
    columns=[
        "LotFrontage",
        "LotArea",
        "MasVnrArea",
        "BsmtFinSF1",
        "BsmtFinSF2",
        "BsmtUnfSF",
#         "TotalBsmtSF",
#         "1stFlrSF",
#         "2ndFlrSF",
#         "GrLivArea",
#         "TotRmsAbvGrd",
#         "GarageYrBlt",
#         "GarageArea",
#         "WoodDeckSF",
#         "OpenPorchSF",
    ],
    diag_kind="kde",
)


## Faceting

## Target variable evaluation

In [ ]:
# evaluate distribution of target variable
machine.eda_transform_target(data=machine.training_target, name=machine.training_target.name)
machine.eda_transform_log1(data=machine.training_target, name=machine.training_target.name)


In [ ]:
# log + 1 transform target
machine.training_target = np.log1p(machine.training_target)


# Data preparation

## Missing data


### Training

In [ ]:
# evaluate missing data
machine.eda_missing_summary(chart_scale=25)

In [ ]:
# missingno matrix
msno.matrix(machine.training_features)

In [ ]:
# missingno bar
msno.bar(machine.training_features)

In [ ]:
# missingno heatmap
msno.heatmap(machine.training_features)

In [ ]:
# missingno dendrogram
msno.dendrogram(machine.training_features)

### Validation

In [ ]:
# evaluate missing data
machine.eda_missing_summary()

In [ ]:
# missingno matrix
msno.matrix(machine.validation_features)

In [ ]:
# missingno bar
msno.bar(machine.validation_features)

In [ ]:
# missingno heatmap
msno.heatmap(machine.validation_features)

In [ ]:
# missingno dendrogram
msno.dendrogram(machine.validation_features)

### Training vs. validation


In [ ]:
# compare feature with missing data
machine.missing_column_compare()

### Impute


In [ ]:
# impute pipeline
category_constant = ['GarageFinish', 'Alley', 'MasVnrType', 'GarageType', 'BsmtFinType1',
                       'BsmtCond', 'BsmtFinType2', 'BsmtQual', 'PoolQC', 'GarageCond',
                       'FireplaceQu', 'GarageQual', 'Fence', 'BsmtExposure', 'MiscFeature']
number_constant = ["GarageYrBlt","MasVnrArea","BsmtUnfSF","GarageArea","BsmtFinSF1","TotalBsmtSF","BsmtFinSF2"]
category_mode = ["Electrical","Functional","SaleType","Exterior1st","MSZoning","Exterior2nd","KitchenQual","Utilities"]
number_mode = ["BsmtHalfBath", "GarageCars", "BsmtFullBath"]

impute_pipe = PandasFeatureUnion([
    ("catConstant", make_pipeline(
        DataFrameSelector(include_columns=category_constant),
        PandasTransformer(SimpleImputer(strategy="constant", fill_value="Nonexistent"))
    )),
    ("numConstant", make_pipeline(
        DataFrameSelector(include_columns=number_constant),
        PandasTransformer(SimpleImputer(strategy="constant", fill_value=0))
    )),
    ("catMode", make_pipeline(
        DataFrameSelector(include_columns=category_mode),
        PandasTransformer(SimpleImputer(strategy="most_frequent"))
    )),
    ("numMode", make_pipeline(
        DataFrameSelector(include_columns=number_mode),
        PandasTransformer(SimpleImputer(strategy="most_frequent"))
    )),
    ("LotFrontage", make_pipeline(
        DataFrameSelector(include_columns=["LotFrontage","Neighborhood"]),
        GroupbyImputer(null_column="LotFrontage", groupby_column="Neighborhood", strategy="mean")
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns=["LotFrontage"] + category_constant + number_constant + category_mode + number_mode),
    )),
])

# fit & save objects
impute_pipe.fit(machine.training_features)
with open(os.path.join(machine.current_experiment_dir, "transformers", "impute_pipe.pkl"), 'wb') as handle:
    pickle.dump(impute_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

# transform datasets
machine.training_features = impute_pipe.fit_transform(machine.training_features)
machine.validation_features = impute_pipe.transform(machine.validation_features)


In [ ]:
#
machine.eda_missing_summary(training_data=True)


In [ ]:
#
machine.eda_missing_summary(training_data=False)


## Feature engineering

### Handcrafted

In [ ]:
# additional features
machine.training_features["BsmtFinSF"] = machine.training_features["BsmtFinSF1"] + machine.training_features["BsmtFinSF2"]
machine.training_features["TotalSF"] = (
    machine.training_features["TotalBsmtSF"] + machine.training_features["1stFlrSF"] + machine.training_features["2ndFlrSF"]
)
machine.update_dtypes()


In [ ]:
# evaluate additional features
for feature in ["BsmtFinSF","TotalSF"]:
    machine.eda_num_target_num_feat(
        feature=feature,
#         outliers_out_of_scope=5,
        chart_scale=20
    )


In [ ]:
# additional features
machine.validation_features["BsmtFinSF"] = machine.validation_features["BsmtFinSF1"] + machine.validation_features["BsmtFinSF2"]
machine.validation_features["TotalSF"] = (
    machine.validation_features["TotalBsmtSF"] + machine.validation_features["1stFlrSF"] + machine.validation_features["2ndFlrSF"]
)


### Polynomial features

In [ ]:
# transform pipe
polynomial_pipe = PandasFeatureUnion([
    ("polynomial", make_pipeline(
        DataFrameSelector(include_columns=["TotalSF","BsmtFinSF"]),
        #DataFrameSelector(include_mlm_dtypes=["continuous"]),
        PandasTransformer(PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)),
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns=["TotalSF","BsmtFinSF"]),
        #DataFrameSelector(exclude_mlm_dtypes=["continuous"]),
    )),
])

# fit & save objects
polynomial_pipe.fit(machine.training_features)
with open(os.path.join(machine.current_experiment_dir, "transformers", "polynomial_pipe.pkl"), 'wb') as handle:
    pickle.dump(polynomial_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

# transform datasets
machine.training_features = polynomial_pipe.fit_transform(machine.training_features)
machine.validation_features = polynomial_pipe.transform(machine.validation_features)

machine.update_dtypes()


### Encoding

#### Evaluate

In [ ]:
# counts of unique values in training data object columns
machine.training_features[machine.training_features.mlm_dtypes["category"]].apply(pd.Series.nunique, axis=0)


In [ ]:
# print unique values in each object columns
machine.unique_category_levels()


In [ ]:
# counts of unique values in validation data string columns
machine.validation_features[machine.validation_features.mlm_dtypes["category"]].apply(pd.Series.nunique, axis=0)


In [ ]:
# print unique values in each object columns
machine.unique_category_levels()


In [ ]:
# identify values that are present in the training data but not the validation data, and vice versa
machine.compare_train_valid_levels()


#### Encode

In [ ]:
# encode pipeline
encode_pipe = PandasFeatureUnion([
    ("nominal", make_pipeline(
        DataFrameSelector(include_columns=nominal),
        PandasTransformer(OneHotEncoder(drop=None, handle_unknown="ignore")),
    )),
    ("ordinal", make_pipeline(
        DataFrameSelector(include_columns=list(ordinal_encodings.keys())),
        PandasTransformer(OrdinalEncoder(categories=list(ordinal_encodings.values()))),
    )),
#     ("bin", make_pipeline(
#         DataFrameSelector(include_columns=machine.training_features.mlm_dtypes["continuous"]),
#         PandasTransformer(KBinsDiscretizer(encode="ordinal")),
#     )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns=nominal + list(ordinal_encodings.keys())),
    )),
])

# fit & save objects
encode_pipe.fit(machine.training_features)
with open(os.path.join(machine.current_experiment_dir, "transformers", "encode_pipe.pkl"), 'wb') as handle:
    pickle.dump(encode_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

# transform datasets
machine.training_features = encode_pipe.fit_transform(machine.training_features)
machine.validation_features = encode_pipe.transform(machine.validation_features)

machine.update_dtypes()


In [ ]:
# target encoding pipe
target_encode_pipe = PandasFeatureUnion([
    ("target", make_pipeline(
        DataFrameSelector(include_mlm_dtypes=["category"]),
        KFoldEncoder(
            target=machine.training_target,
            cv=KFold(n_splits=5, shuffle=False),
            encoder=TargetEncoder,
        ),
    )),
    ("catboost", make_pipeline(
        DataFrameSelector(include_mlm_dtypes=["category"]),
        KFoldEncoder(
            target=machine.training_target,
            cv=KFold(n_splits=5, shuffle=False),
            encoder=CatBoostEncoder,
        ),
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_mlm_dtypes=["category"]),
    )),
])

# fit & save objects
target_encode_pipe.fit(machine.training_features)
with open(os.path.join(machine.current_experiment_dir, "transformers", "target_encode_pipe.pkl"), 'wb') as handle:
    pickle.dump(target_encode_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

# transform datasets
machine.training_features = target_encode_pipe.fit_transform(machine.training_features)
machine.validation_features = target_encode_pipe.transform(machine.validation_features)

machine.update_dtypes()


## Feature transformation

### Skew correction


In [ ]:
# evaluate skew of number features - training data
machine.skew_summary(training_data=True)

In [ ]:
# evaluate skew of number features - validation data
machine.skew_summary(training_data=False)

In [ ]:
# skew correction pipeline
skew_pipe = PandasFeatureUnion([
    ("skew", make_pipeline(
        DataFrameSelector(include_mlm_dtypes=["continuous"]),
        DualTransformer(),
    )),    
    ("diff", make_pipeline(
        DataFrameSelector(exclude_mlm_dtypes=["continuous"]),
    )),
])

# # fit & save objects
# skew_pipe.fit(machine.training_features)
# with open(os.path.join(machine.current_experiment_dir, "transformers", "skew_pipe.pkl"), 'wb') as handle:
#     pickle.dump(skew_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # transform datasets
# machine.training_features = skew_pipe.fit_transform(machine.training_features)
# machine.validation_features = skew_pipe.transform(machine.validation_features)

# machine.update_dtypes()


### Scaling


In [ ]:
#
scale_pipe = PandasFeatureUnion([
    ("scale", make_pipeline(
        DataFrameSelector(),
        PandasTransformer(RobustScaler())
    )),
])

# fit & save objects
scale_pipe.fit(machine.training_features)
with open(os.path.join(machine.current_experiment_dir, "transformers", "scale_pipe.pkl"), 'wb') as handle:
    pickle.dump(scale_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

# transform datasets
machine.training_features = scale_pipe.fit_transform(machine.training_features)
machine.validation_features = scale_pipe.transform(machine.validation_features)

machine.update_dtypes()


## Outliers

In [ ]:
# identify outliers using IQR
train_pipe = Pipeline([
    ("outlier",machine.OutlierIQR(
                outlier_count=20,
                iqr_step=1.5,
                features=machine.training_features.mlm_dtypes["number"],
                drop_outliers=False,))
    ])
machine.training_features = train_pipe.transform(machine.training_features)

# capture outliers
iqr_outliers = np.array(sorted(train_pipe.named_steps["outlier"].outliers))
print(iqr_outliers)

In [ ]:
# identify outliers using Isolation Forest
clf = IsolationForest(
#     behaviour="new",
    max_samples=machine.training_features.shape[0],
    random_state=0,
    contamination=0.01
)
clf.fit(machine.training_features[machine.training_features.columns])
preds = clf.predict(machine.training_features[machine.training_features.columns])

# evaluate index values
mask = np.isin(preds, -1)
if_outliers = np.array(machine.training_features[mask].index)
print(if_outliers)

In [ ]:
# identify outliers using extended isolation forest
train_pipe = Pipeline([
    ("outlier",machine.ExtendedIsoForest(
                columns=machine.training_features.mlm_dtypes["number"],
                n_trees=100,
                sample_size=256,
                extension_level=1,
                anomalies_ratio=0.03,
                drop_outliers=False,))
    ])
machine.training_features = train_pipe.transform(machine.training_features)

# capture outliers
eif_outliers = np.array(sorted(train_pipe.named_steps["outlier"].outliers))
print(eif_outliers)

In [ ]:
# identify outliers that are identified in multiple algorithms
outliers = reduce(np.intersect1d, (iqr_outliers, if_outliers, eif_outliers))
# outliers = reduce(np.intersect1d, (if_outliers, eif_outliers))
print(outliers)

In [ ]:
# review outlier identification summary
outlier_summary = machine.outlier_summary(iqr_outliers=iqr_outliers,
                             if_outliers=if_outliers,
                             eif_outliers=eif_outliers
                            )
outlier_summary[outlier_summary["count"] >= 3]

In [ ]:
# capture index values of known outliers
knownOutliers = (
    machine.training_features[machine.training_features["LotArea"] > 60000].index.values.tolist()
    + machine.training_features[machine.training_features["LotFrontage"] > 300].index.values.tolist()
    + machine.training_features[machine.training_features["GrLivArea"] > 4000].index.values.tolist()
)
knownOutliers = sorted(set(knownOutliers))
print(knownOutliers)

# index of known outliers and outliers identified with the known outliers removed
outliers = [
    53,
    185,
    197,
    437,
    492,
    762,
    796,
    821,
    847,
    1161,
    1221,
    1318,
    1376,
    249,
    313,
    335,
    451,
    523,
    691,
    706,
    934,
    1182,
    1298,
]
# print(outliers)

# remove outlers from predictors and response
# machine.training_features = machine.training_features.drop(outliers)
# machine.training_target = machine.training_target.drop(index=outliers)

print(machine.training_features.shape)
print(machine.training_target.shape)


## Additional exploratory data analysis

In [ ]:
for i in machine.training_features:
    print(i)

## Machine checkpoint

In [ ]:
# save machine object
with open(os.path.join(machine.current_experiment_dir, "machine", "machine.pkl"), 'wb') as handle:
    pickle.dump(machine, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Feature selection

In [ ]:
# generate feature importance summary
estimators = [
    LinearRegression,
    Lasso,
    Ridge,
    ElasticNet,
    KernelRidge,
    SVR,
    LGBMRegressor,
    XGBRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    KNeighborsRegressor,
]

fs = machine.FeatureSelector(
    training_features=machine.training_features,
    training_target=machine.training_target,
    validation_features=machine.validation_features,
    validation_target=machine.validation_target,
    estimators=estimators,
    experiment_dir=machine.current_experiment_dir,
    classification=False,
)
fs.feature_selector_suite(
    sequential_scoring=["root_mean_squared_error"],
    n_jobs=4,
    save_to_csv=True,
    verbose=True,
    run_sfs=False,
    run_sbs=False,    
)


In [ ]:
# calculate cross-validation performance
fs.run_cross_val(
    estimators=estimators,
    scoring=["root_mean_squared_error"],
    n_folds=5,
    step=1,
    n_jobs=2,
    save_to_csv=True,
)


## Mean squared error

In [ ]:
# visualize CV performance for diminishing feature set
fs.plot_results(
    scoring="root_mean_squared_error",
    title_scale=0.8,
    show_features=False,
    marker_on=False,
    save_plots=True
)


In [ ]:
#
fs.create_cross_val_features_df(scoring="root_mean_squared_error")
# fs.cross_val_features_df


In [ ]:
#
fs.create_cross_val_features_dict(scoring="root_mean_squared_error")
# fs.cross_val_features_dict


In [ ]:
# save feature selector
with open(os.path.join(machine.current_experiment_dir, "feature_selection", "FeatureSelector.pkl"), 'wb') as handle:
    pickle.dump(fs, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Modeling

## Data preparation

In [ ]:
#################################################################################
# import training data
df_train, df_valid = data.housing()

continuous = [
    "LotFrontage",
    "LotArea",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "LowQualFinSF",
    "GrLivArea",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch",
    "PoolArea",
]

remove_features = [
    "Id",
    "MiscVal",
]

count = [
    "BsmtFullBath",
    "BsmtHalfBath",
    "FullBath",
    "HalfBath",
    "BedroomAbvGr",
    "KitchenAbvGr",
    "TotRmsAbvGrd",
    "Fireplaces",
    "GarageCars",
]

nominal = [
    "MSSubClass",
    "MSZoning",    
    "LandContour",
    "Neighborhood",
    "Condition1",
    "Condition2",
    "BldgType",
    "HouseStyle",
    "YearBuilt",
    "YearRemodAdd",
    "RoofStyle",
    "RoofMatl",
    "Exterior1st",
    "Exterior2nd",
    "MasVnrType",
    "Foundation",
    "Heating",
    "GarageType",
    "GarageYrBlt", 
    "Fence",
    "SaleType",
    "SaleCondition",
    "MiscFeature",    
    "MoSold",
    "YrSold",
    
]

ordinal = [
    "Street",  
    "Alley" ,
    "LotShape", 
    "Utilities", 
    "LotConfig",
    "LandSlope",
    "OverallQual",
    "OverallCond",    
    "ExterQual", 
    "ExterCond", 
    "BsmtQual", 
    "BsmtCond", 
    "BsmtExposure", 
    "BsmtFinType1", 
    "BsmtFinType2", 
    "HeatingQC", 
    "CentralAir", 
    "Electrical", 
    "KitchenQual", 
    "Functional", 
    "FireplaceQu", 
    "GarageFinish", 
    "GarageQual", 
    "GarageCond", 
    "PavedDrive", 
    "PoolQC",    
]

ordinal_encodings = {
    "Street": ["Grvl", "Pave"],
    "Alley": ["Nonexistent", "Grvl", "Pave"],
    "LotShape": ["IR3", "IR2", "IR1", "Reg"],
    "Utilities": ["ELO", "NoSeWa", "NoSewr", "AllPub"],
    "LotConfig": ["FR3", "FR2", "Corner", "Inside", "CulDSac"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "OverallQual": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "OverallCond": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "ExterQual": ["Po", "Fa", "TA", "Gd", "Ex"], 
    "ExterCond": ["Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtQual": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtCond": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtExposure": ["Nonexistent", "No", "Mn", "Av", "Gd"],
    "BsmtFinType1": ["Nonexistent", "Unf", "LwQ", "BLQ", "Rec", "ALQ", "GLQ"],
    "BsmtFinType2": ["Nonexistent", "Unf", "LwQ", "BLQ", "Rec", "ALQ", "GLQ"],
    "HeatingQC": ["Po", "Fa", "TA", "Gd", "Ex"],
    "CentralAir": ["N", "Y"],
    "Electrical": ["FuseP", "FuseF", "FuseA", "Mix", "SBrkr"],
    "KitchenQual": ["Po", "Fa", "TA", "Gd", "Ex"],
    "Functional": ["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"],
    "FireplaceQu": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "GarageFinish": ["Nonexistent", "Unf", "RFn", "Fin"],
    "GarageQual": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "GarageCond": ["Nonexistent", "Po", "Fa", "TA", "Gd", "Ex"],
    "PavedDrive": ["N", "P", "Y"],
    "PoolQC": ["Nonexistent", "Fa", "TA", "Gd", "Ex"],
}

train = mlm.Machine(
    data=df_train,
    target="SalePrice",
    remove_features=remove_features,
    identify_as_continuous=continuous,
    identify_as_count=count,    
    identify_as_nominal=nominal,
    identify_as_ordinal=ordinal,
    ordinal_encodings=ordinal_encodings,
    is_classification=False,
)

# additional features
machine.training_features["BsmtFinSF"] = machine.training_features["BsmtFinSF1"] + machine.training_features["BsmtFinSF2"]
machine.training_features["TotalSF"] = (
    machine.training_features["TotalBsmtSF"] + machine.training_features["1stFlrSF"] + machine.training_features["2ndFlrSF"]
)

#################################################################################
# import validation data
valid = mlm.Machine(
    data=df_valid,
    remove_features=remove_features,
    identify_as_continuous=continuous,
    identify_as_count=count,    
    identify_as_nominal=nominal,
    identify_as_ordinal=ordinal,
    ordinal_encodings=ordinal_encodings,
    is_classification=False,
)

# change clearly erroneous value to what it probably was
machine.validation_features["GarageYrBlt"].replace({2207: 2007}, inplace=True)

# additional features
machine.validation_features["BsmtFinSF"] = machine.validation_features["BsmtFinSF1"] + machine.validation_features["BsmtFinSF2"]
machine.validation_features["TotalSF"] = (
    machine.validation_features["TotalBsmtSF"] + machine.validation_features["1stFlrSF"] + machine.validation_features["2ndFlrSF"]
)
machine.validation_features.loc[machine.validation_features["TotalSF"].isnull(), "TotalSF"] = (
    machine.validation_features["1stFlrSF"] + machine.validation_features["2ndFlrSF"]
)

machine.update_dtypes()


#################################################################################
# impute pipeline
object_constant = ['GarageFinish', 'Alley', 'MasVnrType', 'GarageType', 'BsmtFinType1',
                       'BsmtCond', 'BsmtFinType2', 'BsmtQual', 'PoolQC', 'GarageCond',
                       'FireplaceQu', 'GarageQual', 'Fence', 'BsmtExposure', 'MiscFeature']
number_constant = ["GarageYrBlt","MasVnrArea","BsmtUnfSF","GarageArea","BsmtFinSF","BsmtFinSF1","TotalBsmtSF","BsmtFinSF2"]
object_mode = ["Electrical","Functional","SaleType","Exterior1st","MSZoning","Exterior2nd","KitchenQual","Utilities"]
number_mode = ["BsmtHalfBath", "GarageCars", "BsmtFullBath"]

impute_pipe = PandasFeatureUnion([
    ("catConstant", make_pipeline(
        DataFrameSelector(object_constant),
        PandasTransformer(SimpleImputer(strategy="constant", fill_value="Nonexistent"))
    )),
    ("numConstant", make_pipeline(
        DataFrameSelector(number_constant),
        PandasTransformer(SimpleImputer(strategy="constant", fill_value=0))
    )),
    ("catMode", make_pipeline(
        DataFrameSelector(object_mode),
        PandasTransformer(SimpleImputer(strategy="most_frequent"))
    )),
    ("numMode", make_pipeline(
        DataFrameSelector(number_mode),
        PandasTransformer(SimpleImputer(strategy="most_frequent"))
    )),
    ("LotFrontage", make_pipeline(
        DataFrameSelector(["LotFrontage","Neighborhood"]),
        GroupbyImputer(null_column="LotFrontage", groupby_column="Neighborhood", strategy="mean")
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns = ["LotFrontage"] + object_constant + number_constant + object_mode + number_mode),
    )),
])

machine.training_features = impute_pipe.fit_transform(machine.training_features)
machine.validation_features = impute_pipe.transform(machine.validation_features)

# #################################################################################
# # polynomial feature pipe
# polynomial_pipe = PandasFeatureUnion([
#     ("polynomial", make_pipeline(
#         DataFrameSelector(include_mlm_dtypes=["continuous"]),
#         PandasTransformer(PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)),
#     )),
#     ("diff", make_pipeline(
#         DataFrameSelector(exclude_mlm_dtypes=["continuous"], exclude_columns=["Name","Cabin"]),
#     )),
# ])

# machine.training_features = polynomial_pipe.fit_transform(machine.training_features)
# machine.validation_features = polynomial_pipe.transform(machine.validation_features)

# machine.update_dtypes()
# 

# feature transformation pipeline
encode_pipe = PandasFeatureUnion([
    ("nominal", make_pipeline(
        DataFrameSelector(include_columns=nominal, exclude_columns=["YearBuilt","GarageYrBlt"]),
        PandasTransformer(OneHotEncoder(drop=None, handle_unknown="ignore")),
    )),
    ("ordinal", make_pipeline(
        DataFrameSelector(include_columns=list(ordinal_encodings.keys())),
        PandasTransformer(OrdinalEncoder(categories=list(ordinal_encodings.values()))),
    )),
    ("bin", make_pipeline(
        DataFrameSelector(include_columns=machine.training_features.mlm_dtypes["continuous"]),
        PandasTransformer(KBinsDiscretizer(encode="ordinal")),
    )),
    ("diff", make_pipeline(
        DataFrameSelector(exclude_columns=nominal + list(ordinal_encodings.keys())),
    )),
])

machine.training_features = encode_pipe.fit_transform(machine.training_features)
machine.validation_features = encode_pipe.transform(machine.validation_features)

machine.update_dtypes()


# # target encoding pipe
# target_encode_pipe = PandasFeatureUnion([
#     ("target", make_pipeline(
#         DataFrameSelector(include_mlm_dtypes=["category"]),
#         KFoldEncoder(
#             target=machine.training_target,
#             cv=KFold(n_splits=5, shuffle=False, random_state=0),
#             encoder=TargetEncoder,
#         ),
#     )),
#     ("catboost", make_pipeline(
#         DataFrameSelector(include_mlm_dtypes=["category"]),
#         KFoldEncoder(
#             target=machine.training_target,
#             cv=KFold(n_splits=5, shuffle=False, random_state=0),
#             encoder=CatBoostEncoder,
#         ),
#     )),
#     ("diff", make_pipeline(
#         DataFrameSelector(exclude_mlm_dtypes=["category"]),
#     )),
# ])

# machine.training_features = target_encode_pipe.fit_transform(machine.training_features)
# machine.validation_features = target_encode_pipe.transform(machine.validation_features)

# machine.update_dtypes()
# 

### scale features
scale_pipe = PandasFeatureUnion([
    ("scale", make_pipeline(
        DataFrameSelector(),
        PandasTransformer(RobustScaler())
    )),
])

machine.training_features = scale_pipe.fit_transform(machine.training_features)
machine.validation_features = scale_pipe.transform(machine.validation_features)

machine.update_dtypes()


#################################################################################
# remove outliers
outliers = [
    53,
    185,
    197,
    437,
    492,
    762,
    796,
    821,
    847,
    1161,
    1221,
    1318,
    1376,
    249,
    313,
    335,
    451,
    523,
    691,
    706,
    934,
    1182,
    1298,
]
machine.training_features = machine.training_features.drop(outliers)
machine.training_target = machine.training_target.drop(index=outliers)

# log transform target
machine.training_target = np.log1p(machine.training_target)
print('completed')

## Bayesian hyper-parameter optimization

In [ ]:
# model/parameter space
estimator_parameter_space = {
    "Lasso": {"alpha": hp.uniform("alpha", 0.0000001, 20)},
    "Ridge": {"alpha": hp.uniform("alpha", 0.0000001, 20)},
    "ElasticNet": {
        "alpha": hp.uniform("alpha", 0.0000001, 20),
        "l1_ratio": hp.uniform("l1_ratio", 0.0, 0.2),
    },
    "KernelRidge": {
        "alpha": hp.uniform("alpha", 0.000001, 15),
        "kernel": hp.choice("kernel", ["linear", "polynomial", "rbf"]),
        "degree": hp.choice("degree", [2, 3]),
        "gamma": hp.uniform("gamma", 0.0, 10),
    },
    "LGBMRegressor": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "boosting_type": hp.choice("boosting_type", ["gbdt", "dart", "goss"])
        # ,'boosting_type': hp.choice('boosting_type'
        #                    ,[{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}
        #                    ,{'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)}
        #                    ,{'boosting_type': 'goss', 'subsample': 1.0}])
        ,
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_child_samples": hp.uniform("min_child_samples", 20, 500),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "num_leaves": hp.uniform("num_leaves", 8, 150),
        "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "subsample_for_bin": hp.uniform("subsample_for_bin", 20000, 400000),
    },
    "XGBRegressor": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "gamma": hp.uniform("gamma", 0.0, 10),
        "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_child_weight": hp.uniform("min_child_weight", 1, 20),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "subsample": hp.uniform("subsample", 0.5, 1),
    },
    "RandomForestRegressor": {
        "bootstrap": hp.choice("bootstrap", [True, False]),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "GradientBoostingRegressor": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "loss": hp.choice("loss", ["ls", "lad", "huber", "quantile"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "SVR": {
        "C": hp.uniform("C", 0.00001, 10),
        "kernel": hp.choice("kernel", ["linear", "poly", "rbf", "sigmoid"]),
        "degree": hp.choice("degree", [2, 3]),
        "gamma": hp.uniform("gamma", 0.0001, 10),
        "epsilon": hp.uniform("epsilon", 0.001, 5),
    },
    "KNeighborsRegressor": {
        "algorithm": hp.choice("algorithm", ["auto", "ball_tree", "kd_tree", "brute"]),
        "n_neighbors": hp.choice("n_neighbors", np.arange(1, 20, dtype=int)),
        "weights": hp.choice("weights", ["distance", "uniform"]),
        "p": hp.choice("p", [1, 2]),
    },
}

bayes_optim_summary = pd.read_csv("bayes_optimization_summary_root_mean_squared_error_2002060358.csv", na_values="nan")

In [ ]:
# execute bayesian optimization grid search
machine.exec_bayes_optim_search(
    estimator_parameter_space=estimator_parameter_space,
    training_features=machine.training_features,
    training_target=machine.training_target,
    validation_features=machine.validation_features,
    validation_target=machine.validation_target,
    scoring="root_mean_squared_error",
    n_folds=5,
    n_jobs=2,
    iters=250,
    show_progressbar=True,
    columns=fs.cross_val_features_dict
)


### Model loss by iteration

In [ ]:
# model loss plot
for estimator in np.unique(machine.bayes_optim_summary["estimator"]):
    machine.model_loss_plot(
        bayes_optim_summary=machine.bayes_optim_summary,
        estimator_class=estimator,
        save_plots=True,
    )


### Parameter selection by iteration

In [ ]:
# estimator parameter plots
for estimator in np.unique(machine.bayes_optim_summary["estimator"]):
    machine.model_param_plot(
        bayes_optim_summary=machine.bayes_optim_summary,
        estimator_class=estimator,
        estimator_parameter_space=estimator_parameter_space,
        n_iter=1000,
#         chart_scale=15,
        title_scale=1.2,
        save_plots=True
    )


## Model performance evaluation - standard models

In [ ]:
#
top_models = machine.top_bayes_optim_models(
                bayes_optim_summary=machine.bayes_optim_summary,
                metric="validation_score",
                num_models=1,
            )
top_models


In [ ]:
bayes_optim_summary["estimator"].unique()

In [ ]:
## standard model fit and predict
# select estimator and iteration
estimator = "LGBMRegressor"; model_iter = 27
# estimator = "XGBRegressor"; model_iter = 20
# estimator = "RandomForestRegressor"; model_iter = 382
# estimator = "GradientBoostingRegressor"; model_iter = 238
# estimator = "SVR"; model_iter = 259

# extract params and instantiate model
model = machine.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary, estimator_class=estimator, model_iter=model_iter
)
model.fit(machine.training_features.values, machine.training_target.values)

X_train, X_valid, y_train, y_valid = train_test_split(machine.training_features, machine.training_target)
y_pred = model.predict(machine.training_features.values)

In [ ]:
machine.regression_panel(
    model=model,
    X_train=machine.training_features,
    y_train=machine.training_target,
)

In [ ]:
machine.regression_panel(
    model=model,
    X_train=machine.training_features,
    y_train=machine.training_target,
#     X_train=X_train,
#     y_train=y_train,
#     X_valid=X_valid,
#     y_valid=y_valid,
#     n_folds=4,
)

In [ ]:
df = machine.regression_prediction_summary(
    model=model,
    X_train=machine.training_features,
    y_train=machine.training_target,
)
df

# Model explanability

In [ ]:
# 
# estimator = "XGBRegressor"; model_iter = 418
estimator = "LGBMRegressor"; model_iter = 27

model = machine.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary, estimator=estimator, model_iter=model_iter
)

model.fit(machine.training_features.values, machine.training_target.values)

## Permutation importance

## Partial dependence plots

## SHAP

### Training

In [ ]:
# SHAP force plots for individual observations
for i in machine.training_features.index[:5]:
    machine.single_shap_viz_tree(obs_ix=i, model=model, data=machine.training_features, target=machine.training_target, classification=False)

In [ ]:
# SHAP force plot a set of data
visual = machine.multi_shap_viz_tree(obs_ixs=machine.training_features.index, model=model, data=machine.training_features)
visual

In [ ]:
# generate SHAP values for set of observations
obs_data, _, obs_shap_values = machine.multi_shap_value_tree(
    obs_ixs=machine.training_features.index, model=model, data=machine.training_features
)

In [ ]:
# SHAP dependence plot grid
grid_features = ["OverallCond","LotFrontage","TotalSF","BsmtFinSF","LotConfig"]

machine.shap_dependence_grid(
    obs_data=obs_data,
    obs_shap_values=obs_shap_values,
    grid_features=grid_features,
    all_features=machine.training_features.columns,
    dot_size=35,
    alpha=0.5,
)

In [ ]:
# single SHAP dependence plot
p = PrettierPlot()
ax = p.make_canvas()

machine.shap_dependence_plot(
    obs_data=obs_data,
    obs_shap_values=obs_shap_values,
    scatter_feature="TotalSF",
    color_feature="LotFrontage",
    feature_names=machine.training_features.columns.tolist(),
    dot_size=50,
    alpha=0.5,
    ax=ax    
)

In [ ]:
# SHAP dependence plots for all feature relative to an interaction feature
feature_names = machine.training_features.columns.tolist()
top_shap = np.argsort(-np.sum(np.abs(obs_shap_values), 0))

# generate force plot
for top_ix in top_shap:
    p = PrettierPlot()
    ax = p.make_canvas()
    
    machine.shap_dependence_plot(
        obs_data=obs_data,
        obs_shap_values=obs_shap_values,
        scatter_feature=feature_names[top_ix],
        color_feature="OverallCond_ordinal_encoded",
        feature_names=feature_names,
        dot_size=35,
        alpha=0.5,
        ax=ax
    )

In [ ]:
# SHAP summary plot
feature_names = machine.training_features.columns.tolist()
machine.shap_summary_plot(
        obs_data=obs_data,
        obs_shap_values=obs_shap_values,
        feature_names=feature_names,
    )


### Validation

In [ ]:
# SHAP force plots for individual observations
for i in machine.validation_features.index[:5]:
    machine.single_shap_viz_tree(obsIx=i, model=model, data=machine.validation_features, classification=False)

In [ ]:
# SHAP force plot a set of data
visual = machine.multi_shap_viz_tree(obs_ixs=machine.validation_features.index, model=model, data=machine.validation_features)
visual

In [ ]:
# generate SHAP values for set of observations
obs_data, _, obs_shap_values = machine.multi_shap_value_tree(
    obs_ixs=machine.validation_features.index, model=model, data=machine.validation_features
)

In [ ]:
# SHAP dependence plot grid
grid_features = ["OverallCond","LotFrontage","TotalSF","BsmtFinSF","LotConfig"]

machine.shap_dependence_grid(
    obs_data=obs_data,
    obs_shap_values=obs_shap_values,
    grid_features=grid_features,
    all_features=machine.validation_features.columns,
    dot_size=35,
    alpha=0.5,
)

In [ ]:
# single SHAP dependence plot
p = PrettierPlot()
ax = p.make_canvas()

machine.shap_dependence_plot(
    obs_data=obs_data,
    obs_shap_values=obs_shap_values,
    scatter_feature="TotalSF",
    color_feature="LotFrontage",
    feature_names=machine.validation_features.columns.tolist(),
    dot_size=50,
    alpha=0.5,
    ax=ax    
)

In [ ]:
# SHAP dependence plots for all feature relative to an interaction feature
feature_names = machine.validation_features.columns.tolist()
top_shap = np.argsort(-np.sum(np.abs(obs_shap_values), 0))

# generate force plot
for top_ix in top_shap:
    p = PrettierPlot()
    ax = p.make_canvas()
    
    machine.shap_dependence_plot(
        obs_data=obs_data,
        obs_shap_values=obs_shap_values,
        scatter_feature=feature_names[top_ix],
        color_feature="Age",
        feature_names=feature_names,
        dot_size=35,
        alpha=0.5,
        ax=ax
    )

In [ ]:
# SHAP summary plot
feature_names = machine.validation_features.columns.tolist()
machine.shap_summary_plot(
        obs_data=obs_data,
        obs_shap_values=obs_shap_values,
        feature_names=feature_names,
    )


# Save objects

In [ ]:
# 
with open(os.path.join(machine.current_experiment_dir, "machine", "machine.pkl"), 'wb') as handle:
    pickle.dump(machine, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(machine.current_experiment_dir, "transformers", "impute_pipe.pkl"), 'wb') as handle:
    pickle.dump(impute_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(machine.current_experiment_dir, "transformers", "polynomial_pipe.pkl"), 'wb') as handle:
    pickle.dump(polynomial_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(machine.current_experiment_dir, "transformers", "encode_pipe.pkl"), 'wb') as handle:
    pickle.dump(encode_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(machine.current_experiment_dir, "transformers", "target_encode_pipe.pkl"), 'wb') as handle:
    pickle.dump(target_encode_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(os.path.join(machine.current_experiment_dir, "transformers", "skew_pipe.pkl"), 'wb') as handle:
#     pickle.dump(skew_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(machine.current_experiment_dir, "transformers", "scale_pipe.pkl"), 'wb') as handle:
    pickle.dump(scale_pipe, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(machine.current_experiment_dir, "feature_selection", "FeatureSelector.pkl"), 'wb') as handle:
    pickle.dump(fs, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Stacking

## Primary models

In [ ]:
# get out-of-fold predictions
oof_train, oof_valid, columns = machine.model_stacker(
    models=top_models,
    bayes_optim_summary=bayes_optim_summary,
    X_train=machine.training_features.values,
    y_train=machine.training_target.values,
    X_valid=machine.validation_features.values,
    n_folds=10,
    n_jobs=10,
)

In [ ]:
# view correlations of predictions
p = PrettierPlot()
ax = p.make_canvas()
p.corr_heatmap(
    df=pd.DataFrame(oof_train, columns=columns), annot=True, ax=ax, vmin=0
)

## Meta model

In [ ]:
# model/parameter space
estimator_parameter_space = {
    "KernelRidge": {
        "alpha": hp.uniform("alpha", 0.000001, 15),
        "kernel": hp.choice("kernel", ["linear", "polynomial", "rbf"]),
        "degree": hp.choice("degree", [2, 3]),
        "gamma": hp.uniform("gamma", 0.0, 10),
    },
    "LGBMRegressor": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "boosting_type": hp.choice("boosting_type", ["gbdt", "dart", "goss"])
        # ,'boosting_type': hp.choice('boosting_type'
        #                    ,[{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}
        #                    ,{'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)}
        #                    ,{'boosting_type': 'goss', 'subsample': 1.0}])
        ,
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_child_samples": hp.uniform("min_child_samples", 20, 500),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "num_leaves": hp.uniform("num_leaves", 8, 150),
        "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "subsample_for_bin": hp.uniform("subsample_for_bin", 20000, 400000),
    },
    "XGBRegressor": {
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0),
        "gamma": hp.uniform("gamma", 0.0, 10),
        "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
        "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "min_child_weight": hp.uniform("min_child_weight", 1, 20),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "subsample": hp.uniform("subsample", 0.5, 1),
    },
    "RandomForestRegressor": {
        "bootstrap": hp.choice("bootstrap", [True, False]),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "GradientBoostingRegressor": {
        "n_estimators": hp.choice("n_estimators", np.arange(100, 10000, 10, dtype=int)),
        "max_depth": hp.choice("max_depth", np.arange(2, 20, dtype=int)),
        "max_features": hp.choice("max_features", ["auto", "sqrt"]),
        "learning_rate": hp.uniform("learning_rate", 0.000001, 0.2),
        "loss": hp.choice("loss", ["ls", "lad", "huber", "quantile"]),
        "min_samples_split": hp.choice(
            "min_samples_split", np.arange(2, 40, dtype=int)
        ),
        "min_samples_leaf": hp.choice("min_samples_leaf", np.arange(2, 40, dtype=int)),
    },
    "SVR": {
        "C": hp.uniform("C", 0.00001, 10),
        "kernel": hp.choice("kernel", ["linear", "poly", "rbf", "sigmoid"]),
        "degree": hp.choice("degree", [2, 3]),
        "gamma": hp.uniform("gamma", 0.0001, 10),
        "epsilon": hp.uniform("epsilon", 0.001, 5),
    },
}

In [ ]:
# execute bayesian optimization grid search
machine.exec_bayes_optim_search(
    estimator_parameter_space=estimator_parameter_space,
    results_dir="{}_hyperopt_meta_{}.csv".format(rundate, analysis),
    X=oof_train,
    y=machine.training_target,
    scoring="accuracy",
    n_folds=8,
    n_jobs=10,
    iters=1000,
    verbose=0,
)

In [ ]:
# read scores summary table
analysis = "housing"
rundate = "20190807"
bayes_optim_summary_meta = pd.read_csv("{}_hyperopt_meta_{}.csv".format(rundate, analysis))
bayes_optim_summary_meta[:5]

In [ ]:
# model loss plot
for estimator in np.unique(bayes_optim_summary_meta["estimator"]):
    machine.model_loss_plot(bayes_optim_summary=bayes_optim_summary_meta, estimator=estimator)

In [ ]:
# estimator parameter plots
for estimator in np.unique(bayes_optim_summary_meta["estimator"]):
    machine.modelParamPlot(
        bayes_optim_summary=bayes_optim_summary_meta,
        estimator=estimator,
        estimator_parameter_space=estimator_parameter_space,
        n_iter=100,
        chart_scale=15,
    )

## Model performance evaluation - stacked models

In [ ]:
top_models = machine.top_bayes_optim_models(
    bayes_optim_summary=bayes_optim_summary_meta, num_models=1
)
top_models

In [ ]:
# best second level learning model
estimator = "LGBMClassifier"; model_iter = 668
# estimator = "XGBClassifier"; model_iter = 380
# estimator = "RandomForestClassifier"; model_iter = 411
# estimator = "GradientBoostingClassifier"; model_iter = 590
# estimator = "SVC"; model_iter = 135

# extract params and instantiate model
model = machine.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary_meta, estimator=estimator, model_iter=model_iter
)

# single model evaluation here

In [ ]:
# ,multi model evaluation here

## Submission - stacked models

In [ ]:
# best second level learning model
estimator = "LGBMClassifier"; model_iter = 668
# estimator = "XGBClassifier"; model_iter = 380
# estimator = "RandomForestClassifier"; model_iter = 411
# estimator = "GradientBoostingClassifier"; model_iter = 590
# estimator = "SVC"; model_iter = 135

# extract params and instantiate model
model = machine.BayesOptimModelBuilder(
    bayes_optim_summary=bayes_optim_summary_meta, estimator=estimator, model_iter=model_iter
)

model.fit(oof_train, machine.training_target.values)
y_pred = model.predict(oof_valid)
# print(sum(y_pred))

In [ ]:
# generate prediction submission file
submit = pd.DataFrame({"Id": df_test.Id, "SalePrice": np.expm1(y_pred)})
submit.to_csv("data/submission.csv", index=False)